In [1]:
import sys
sys.path.append("../")

In [2]:
from db import get_connection
from typing import List, Tuple
class AudioHashh:
  def __init__(self):
    pass

  @classmethod
  def create_table(cls):
    conn = get_connection()
    sql = """
    CREATE TABLE IF NOT EXISTS audio_hash (
        id serial PRIMARY KEY,
        key TEXT NOT NULL,
        minhash TEXT NOT NULL,
        name TEXT NOT NULL,
        frame INTEGER NOT NULL  
      )
    """
    conn.cursor().execute(sql)
    conn.commit()
  
  def insertMany(self, data):
    conn = get_connection()
    sql = "INSERT INTO audio_hash (id, key, minhash, name, frame) VALUES (DEFAULT, %s, %s, %s, %s)"
    conn.cursor().executemany(sql, data)
    conn.commit()
    
  def getMinHash(self, band)->List[Tuple[str,str,str,int]]:
    conn = get_connection()
    sql = """
    SELECT key, minhash, name, frame FROM audio_hash WHERE key in %s
    """
    cur = conn.cursor()
    cur.execute(sql, (band,))
    minhash = cur.fetchall()
    cur.close()
    conn.close()
    return minhash

In [3]:
import numpy as np
import random
from models.audio.AudioHashModel import AudioHash

class MinHash:
  def __init__(self, num_perm):
    self.permutations = self._permutations_generate(num_perm)
  
  def _permutations_generate(self, num_perm, seed=5421)->list:
    np.random.seed(seed)
    permutations = [np.random.choice( 255, 100,replace=False) for _ in range(num_perm)]
    return permutations
      
  def hash(self,array)->list:
    signature = []
    for permutation in self.permutations:
      for j in permutation:
        if(array[j] > 0):
          signature.append(j)
          break
    return signature
  
  def jaccard_similarity(self,signature1, signature2):
    intersection = np.sum(np.bitwise_and(signature1, signature2))
    union = np.sum(np.bitwise_or(signature1, signature2))
    return intersection / union 

class LsH:
    def __init__(self, threshold:float,num_hash_functions:int = 100,bands:int = 25, rows:int = 4):
      assert threshold > 0 and threshold < 1
      assert bands*rows == num_hash_functions
      self._threshold = threshold
      self._num_hash_functions = num_hash_functions
      self._bands = bands
      self._rows = rows
      self._min_hash = MinHash(num_hash_functions)
      self._buckets = {}
    
    def clear(self):
      self._buckets.clear()
    
    def searchInDb(self, fingerprints,audioHashes:AudioHash,name,time):
        min_hashes = self._min_hash.hash(fingerprints)
        band_hashes = []
        compared_sketches = set()
        
        if(len(min_hashes) != self._num_hash_functions):
          return None
        
        
        for i in range(self._bands):
          band_hashes.append(self._compute_band_hash(min_hashes, i))
        dbHashes = audioHashes.getMinHash(tuple(band_hashes))
        
        for dbHashTuple in dbHashes:
          if dbHashTuple[0] in compared_sketches:
            continue
          dbHash = list(map(int, dbHashTuple[1].split(",")))
          jac = self._min_hash.jaccard_similarity(dbHash, min_hashes)
          if jac >= self._threshold:
            return (time, dbHashTuple[0], dbHashTuple[2], dbHashTuple[3], jac)
          compared_sketches.add(dbHashTuple[0])
          
        
        # for i in range(self._bands):
        #   band_hashes.append(self._compute_band_hash(min_hashes, i))
        #   hashes = audioHashes.getMinHash(band_hashes[i])
        #   for hash in hashes:
        #     if(band_hashes[i] not in compared_sketches):
        #       hash_to_check = list(map(int, hash[0].split(",")))
        #       jac = self._min_hash.jaccard_similarity(hash_to_check, min_hashes)
        #       if jac >= self._threshold:
        #         return (time, hash[1], hash[2], jac)
        #       compared_sketches.add(band_hashes[i])
            
          # if band_hashes[i] in self._buckets:
          #   for sketch_to_check in self._buckets[band_hashes[i]]:
          #     check_key = ''.join(str(x) for x in sketch_to_check)
          #     if check_key not in compared_sketches:
          #       jac = self._min_hash.jaccard_similarity(sketch_to_check[0], min_hashes)
          #       if jac >= self._threshold:
          #         return (sketch_to_check[0],sketch_to_check[1],sketch_to_check[2], jac)
          #       compared_sketches.add(check_key)
    
    def search(self, fingerprints,name,time, add_to_bucket = True)->tuple :
        min_hashes = self._min_hash.hash(fingerprints)
        band_hashes = []
        compared_sketches = set()
        
        if(len(min_hashes) != self._num_hash_functions):
          return None
        
        
        for i in range(self._bands):
          band_hashes.append(self._compute_band_hash(min_hashes, i))
          if band_hashes[i] in self._buckets:
            for sketch_to_check in self._buckets[band_hashes[i]]:
              check_key = ''.join(str(x) for x in sketch_to_check)
              if check_key not in compared_sketches:
                jac = self._min_hash.jaccard_similarity(sketch_to_check[0], min_hashes)
                if jac >= self._threshold:
                  return (sketch_to_check[0],sketch_to_check[1],sketch_to_check[2], jac)
                compared_sketches.add(check_key)
        if add_to_bucket and len(min_hashes) == self._num_hash_functions:
          for i in range(self._bands):
            if band_hashes[i] not in self._buckets:
              self._buckets[band_hashes[i]] = []
            self._buckets[band_hashes[i]].append((min_hashes, name, time))
            
    def add(self, fingerprints, name, time):
        min_hashes = self._min_hash.hash(fingerprints)
        band_hashes = []
        
        if(len(min_hashes) != self._num_hash_functions):
          return None
        
        for i in range(self._bands):
          band_hashes.append(self._compute_band_hash(min_hashes, i))
          if band_hashes[i] not in self._buckets:
            self._buckets[band_hashes[i]] = []
          self._buckets[band_hashes[i]].append((min_hashes, name, time))
          
    def _compute_band_hash(self, min_hashes: list, i: int) -> str:
        """Compute a hash for quick bucket match search."""
        band_hash_list = []
        for j in range(self._rows):
            # Adding the rows corresponding to ith band
            band_hash_list.append('%02d' % min_hashes[i * self._rows + j])

        # Adding the number i to distinguish between bands
        band_hash_list.append('%02d' % i)
        return ''.join(band_hash_list)
      
    def get_min_hash(self):
      return self._min_hash
      

In [4]:
from libs.ContentDataSet import ContentDataset

In [5]:
valDataset = ContentDataset("../data/compressed_val/")

In [6]:
licDataset = ContentDataset("../data/test/")

In [7]:
audioLSH = LsH(threshold=0.81)

In [8]:
videoLSH = LsH(threshold=0.65)

In [9]:
audio1 = valDataset.content_list[0][1]
audio2 = licDataset.content_list[0][1]

In [10]:
audio1Spec = valDataset.content_list[0][3]

In [11]:
audio1.__len__()

535

In [12]:
video1 = valDataset.content_list[0][2]
video2 = licDataset.content_list[0][2]

In [13]:
for k,f in enumerate(audio2):
  audioLSH.search(f,"Licence",k)

In [14]:
def getSpecificFingerprint(mel_spec,start,end):
  def get_energy_spec(spec):
    # Transpose 'spec' upfront and get the shifted versions for calculations
    specT = spec.T
    specT_shifted_vertically = np.roll(specT, shift=-1, axis=0)
    specT_shifted_horizontally = np.roll(specT, shift=-1, axis=1)

    # Calculate the differences for current and previous rows
    diff_current = specT - specT_shifted_horizontally
    diff_previous = specT_shifted_vertically - np.roll(specT_shifted_horizontally, shift=-1, axis=0)

    # Compute the energy_spec matrix using NumPy's greater function for element-wise comparison
    energy_spec = np.greater(diff_current - diff_previous, 0).astype(int)
    return energy_spec.T
  def get_audio_fingerprint(spec):
    fingerprint = []
    energy_spec = get_energy_spec(spec)
    for i in range(0,energy_spec.shape[1],128):
      fingerprint.append(energy_spec[:,i:i+128].flatten())
    return fingerprint 
  return get_audio_fingerprint(mel_spec[:,start:end])
  
   

In [15]:
fragments =[]
for k,f in enumerate(audio1):
  res = audioLSH.search(f,"Pirate",k,add_to_bucket = False)
  if res != None:
    fragments.append((k,res[0],res[1],res[2],res[3]))
    print(k*1.48/60,res[1],res[2]*1.48/60,res[3])

4.045333333333334 Licence 12.382666666666667 0.8110370265285857
6.487333333333334 Licence 1.2826666666666666 0.8257506227874656
8.288 Licence 8.164666666666667 0.8255813953488372
9.842 Licence 14.06 0.8214086859688196


In [21]:
minHash = audioLSH.get_min_hash()
#10 sec interval in our frames
interval = 30
for i, v in enumerate(fragments):
  start = int(v[0]) - interval
  end = int(v[0]) + interval
  if start < 0:
    start = 0
  #audio1 is fingerprint of the original audio
  if end > len(audio1):
    end = len(audio1)
  
  print(start,end)
  for j in range(start,end):
    newFingerprint = getSpecificFingerprint(audio1Spec,j,end)
    for k,f in enumerate(newFingerprint[:-1]):
      res = audioLSH.search(f,"Pirate",k,add_to_bucket = False)
      if res != None:
        fragments.append((k,res[0],res[1],res[2],res[3]))
        print(k*1.48/60,res[1],res[2]*1.48/60,res[3])
  

134 194
233 293
306 366
369 429


In [17]:
for k,f in enumerate(video2):
  videoLSH.search(f,"Licence",k)

In [18]:
for k,f in enumerate(video1):
  res = videoLSH.search(f,"Pirate",k,add_to_bucket = False)
  if res != None:
    print(k*2/60,res[1],res[2]*2/60,res[3])

0.5666666666666667 Licence 15.133333333333333 0.7108605825638933
0.6 Licence 15.133333333333333 0.7151924863246981
0.9666666666666667 Licence 0.7 0.6741239238510973
1.9666666666666666 Licence 4.466666666666667 0.671400449943757
2.2333333333333334 Licence 25.0 0.7140940317732932
2.3333333333333335 Licence 2.5 0.6628329297820823
3.2333333333333334 Licence 0.0 1.0
4.4 Licence 4.233333333333333 0.6679185232336091
5.766666666666667 Licence 14.033333333333333 0.7032667876588021
6.533333333333333 Licence 7.1 0.6870778083562672
6.666666666666667 Licence 0.13333333333333333 0.6673517322372284
7.233333333333333 Licence 10.4 0.6508113790158334
8.9 Licence 0.0 1.0
9.1 Licence 20.866666666666667 0.6877225866916589
10.966666666666667 Licence 22.8 0.7039084738340267
11.366666666666667 Licence 22.633333333333333 0.6517304877472968
12.933333333333334 Licence 2.5 0.6807165437302424
